### 분할 이미지 데이터 모으기

In [1]:
# !pip install tqdm
from shutil import copyfile
from glob import glob
from tqdm import tqdm
import os

In [2]:
def movefiles(filepath, movepath):
    copyfile(filepath, movepath)

In [3]:
train_paths = glob(os.path.join('data', 'train', '*', '*.jpg'))
test_paths = glob(os.path.join('data', 'test', '*', '*.jpg'))

In [4]:
len(train_paths), len(test_paths)

(95454, 8000)

In [5]:
train_rename = ['data\\train_all\\' + train_path.split('\\')[-1] for train_path in train_paths]
test_rename = ['data\\test_all\\' + test_path.split('\\')[-1] for test_path in test_paths]

In [6]:
len(train_rename), len(test_rename)

(95454, 8000)

In [7]:
print(train_paths[0], train_rename[0])

data\train\0\0017qu55dk.jpg data\train_all\0017qu55dk.jpg


In [8]:
for i in tqdm(range(len(train_paths))):
    copyfile(train_paths[i], train_rename[i])


100%|██████████| 95454/95454 [24:18<00:00, 65.43it/s]


In [9]:
for i in tqdm(range(len(test_paths))):
    copyfile(test_paths[i], test_rename[i])

100%|██████████| 8000/8000 [01:04<00:00, 124.80it/s]


## Using Colab
##### - 통합 이미지 데이터 구글 업로드 및 압축 풀기

In [ ]:
import pandas as pd
import numpy as np

In [1]:
# !unzip /content/drive/"My Drive"/Dacon/train_all.zip -d /content/drive/"My Drive"/Dacon/train/
# !unzip /content/drive/"My Drive"/Dacon/test_all.zip -d /content/drive/"My Drive"/Dacon/test/

In [ ]:
train_csv = '/content/drive/My Drive/Dacon/train.csv'
sample_sub_csv = '/content/drive/My Drive/Dacon/sample_submission.csv'

In [ ]:
train = pd.read_csv(train_csv)
sample = pd.read_csv(sample_sub_csv)

In [ ]:
# CategoryId No.11 = Leggings 존재 X
# train['CategoryId'].value_counts()

In [ ]:
# colab gpu 환경에 이미 설치가 되어있다. 그 이외에 것만 따로 설치.
# # install dependencies: (use cu101 because colab has CUDA 10.1)
# !pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

# # Install mmdetection
# !rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

# !pip install -e .

# # install Pillow 7.0.0 back in order to avoid bug in colab
# !pip install Pillow==7.0.0

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation 
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
!mkdir checkpoints
!wget -c https://open-mmlab.s3.ap-northeast-2.amazonaws.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth \
      -O checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth

In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# Choose to use a config and initialize the detector
config = 'configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco.py'
# Setup a checkpoint file to load
checkpoint = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'
# initialize the detector
model = init_detector(config, checkpoint, device='cuda:0')

In [ ]:
# 위 코드 오류 발생 및 해결 방법
# ArributeError: No module named 'terminaltables'
!pip install terminaltables tqdm
# ArributeError: module 'pycocotools' has no attribute '__version__'